In [52]:
'''

2019. 11. 28.

by Yumi Kim CBBL at GNU
Screening file and Suggest Candidates which has good activity values

'''

import pandas as pd
import numpy as np
import pymysql.cursors

file_name_KCB_data = 'KCB_data_157694_update.csv'
data_df = pd.read_csv(file_name_KCB_data)

file_name_descriptor_list = 'qsar_descriptor_list.csv'
descriptor_list_df = pd.read_csv(file_name_descriptor_list)

file_name_all_qsar_df = 'qsar_model_list.csv'
all_qsar_df = pd.read_csv(file_name_all_qsar_df)

input_IC50_value = 0.5
screening_output_file_name = 'screening_result.csv'



def remove_specific_char(origine_srt, specific_char):
    return ''.join( c for c in origine_srt if  c not in specific_char )


def get_weight_list(weights):
    weights = remove_specific_char(weights, '[]')
    weights = weights.split(" ")

    weight_list = []
    
    for i in range(len(weights)):
        if(len(weights[i]) != 0):
            weight_list.append(float(weights[i]))
    
    return weight_list

# Get QSAR Weights and bias
bias = all_qsar_df['bias_vals'].values[0]
bias = remove_specific_char(bias, '[]')
bias = float(bias)
weights = all_qsar_df['weights_vals'].values[0]

weight_list = get_weight_list(weights)

des_list = ['smile','corp_id']
des_list_except_smile = []
for i in range(descriptor_list_df.shape[0]):
    des_list.append(descriptor_list_df['0'][i])
    des_list_except_smile.append(descriptor_list_df['0'][i])
    
optimized_df = data_df[des_list]

optimized_df["Pred_IC50"] = 0
for i in range(len(weight_list)):
    print(optimized_df[des_list_except_smile[i]]*weight_list[i])
    optimized_df["Pred_IC50"] += optimized_df[des_list_except_smile[i]]*weight_list[i]
optimized_df["Pred_IC50"]+= bias

optimized_df = optimized_df.sort_values(["Pred_IC50"], ascending=[True])
# Selection with Condition
low_value = optimized_df['Pred_IC50'] > 0
high_value = optimized_df['Pred_IC50'] < input_IC50_value
same_bias = optimized_df['Pred_IC50'] != bias
suggest_compounds_df = optimized_df[low_value & high_value & same_bias]
suggest_compounds_df.to_csv(screening_output_file_name, index=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1144,1145,1223,1224,1225,1226,1227,1228) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0       -97.960291
1       -66.981283
2        -9.160850
3       -26.583651
4       -64.621670
5       -18.579474
6       -23.397843
7        43.557002
8       -55.897050
9       -72.447389
10       54.317366
11       -4.930732
12       17.436020
13       47.555785
14      -36.233609
15       -9.630129
16       -0.727052
17       42.314404
18       38.864214
19        6.926819
20      -49.492387
21      -43.206695
22      -14.296479
23      -99.539976
24      -47.806949
25      -11.586559
26      -36.524430
27      -81.409953
28       19.478374
29      -10.707487
           ...    
8904     69.611888
8905     67.893402
8906     31.296267
8907    -78.072125
8908     97.828100
8909     17.581430
8910     32.360407
8911     -5.135628
8912    -25.625265
8913     -5.294258
8914     56.412596
8915     35.414023
8916     43.140599
8917   -130.578471
8918   -197.097084
8919   -135.568689
8920    -55.407943
8921   -198.670160
8922    -84.304940
8923    102.501059
8924    -37.178776
8925    -93.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [53]:
suggest_compounds_df

,smile,corp_id,ALogP,AATS4m,AATS4v,C1SP3,LipoaffinityIndex,Pred_IC50
3480,O=C(OC(C)(C)C)[C@](C)([H])N/C(=N/O)C=1C=CC(=CC...,5523,0.1920,48.261615,145.848467,4,5.667123,0.376480
7958,O=[S](=O)(NC(=O)N/C1=N/C(=CC(=N1)OC)OC)C2=C/C=...,12327,-1.2619,88.304431,190.975724,2,4.641286,0.437034
